In [1]:
###libs
import os
import argparse
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import cv2 
###files
from config import Config as cfg
from dataProcessing import VOC_dataset as dataset
from models.network import NetAPI
from trainer import Trainer
from utils import tensor_to_img


In [ ]:
def draw_bboxes(img,bboxes,color,th=1):
    img_ = img.copy()
    for bbox in bboxes:
        x,y,w,h = bbox
        pt1 = (int(x-w/2),int(y-h/2))
        pt2 = (int(x+w/2),int(y+h/2))
        img_ =cv2.rectangle(img_,pt1,pt2,color,thickness = th)
    return img_
def draw_bboxes_wt_text(img,bboxes,color,th=1):
    img_ = img.copy()
    for bbox in bboxes:
        x,y,w,h = bbox
        pt1 = (int(x-w/2),int(y-h/2))
        pt2 = (int(x+w/2),int(y+h/2))
        img_ =cv2.rectangle(img_,pt1,pt2,color,thickness = th)
    return img_

In [2]:
exp = 'debug'
resume = 'best'
net = 'yolo'

In [3]:
config  = cfg(mode='val')
val_set = dataset(config)
val_set = dataset(config,mode='val')
val_bs = 2
val_loader = DataLoader(val_set,batch_size=val_bs,shuffle=False,pin_memory=False,collate_fn=val_set.collate_fn)
datasets = {'val':val_loader}

config.device = torch.device("cuda")
torch.cuda.empty_cache()
#for reproducity
torch.manual_seed(2333)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [4]:
network = NetAPI(config,net,'yolo')


75 72 3 61678657
finish load from path: ../network_weights/yolov3.weights


In [5]:
config.exp_name = exp
det = Trainer(config,datasets,network,(resume,-1))

load:../checkpoints/debug/epoch_best.pt


In [6]:
imgs,pds,gts,sizes = det.validate_random()
print(imgs[0].shape)

torch.Size([3, 416, 416])


In [ ]:
for b in range(val_bs):
    pd = pds[b]
    gt = gts[b]
    size = sizes[b]
    img = cv2.resize((tensor_to_img(imgs[b]),size)
    plt.imshow(img)